In [1]:
import os
import numpy as np
import random
from scipy import ndimage
from skimage import transform
import keras
from keras.layers import Dense
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import skimage.io as io

# MEAN = np.array([123.0, 117.0, 104.0])
MEAN = np.array([0, 0, 0])
def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-2].output #Last FC layer's output  
    softmax_layer = Dense(256, activation='softmax')(vgg_out) #Create softmax layer taking input as vgg_ou
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate
    for l in vgg_model.layers:
        l.trainable = False

    return tl_model
def convert_image(collect):
#     sz = np.shape(collect)[0]
    sz = len(collect)
    images = np.zeros((sz, 224, 224, 3))
    for i in range(sz):
        img = collect[i]
        img = transform.resize(img, (224, 224))*255
        if (len(img.shape) != 3):
           img = img[:,:,np.newaxis]
        images[i] = img
    return images

sizeOfEachClass = 32
sizeOfTestEachClass = 16
path='/home/yiluo/Homework/253/3/256_ObjectCategories/'
files = os.listdir(path)
X_train, X_val, X_test = [], [], []
Y_train, Y_val, Y_test = [], [], []
label = 0
for category in files:
    sub_path = path + category
    sub_files = os.listdir(sub_path)
    cnt = np.shape(sub_files)[0]
    num = 1
    for img_nm in sub_files:
        if img_nm[-4:] == '.jpg':
            img_path = sub_path + '/' + img_nm
            #img = ndimage.imread(sub_path + '/' + img_nm)
            #img = transform.resize(img, (224, 224))*255
            if num <= np.floor(0.8*cnt):
                if num <= sizeOfEachClass:
                    X_train.append(img_path)
                    Y_train.append(label)
            elif num <= np.floor(0.9*cnt):
                if num <= np.floor(0.8*cnt) + sizeOfTestEachClass:
                    X_val.append(img_path)
                    Y_val.append(label)
            else:
                X_test.append(img_path)
                Y_test.append(label)
            num += 1
    label += 1

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [3]:
#shuffle
ind=range(int(len(X_train)))
random.shuffle(ind)
X_train = [X_train[i] for i in ind]
Y_train = [Y_train[i] for i in ind]
Y_train_label = np.zeros((len(Y_train), 256))
for i in range(len(Y_train)):
    Y_train_label[i, Y_train[i]] = 1
    
#validation  
Y_val_label = np.zeros((len(Y_val), 256))
for i in range(len(Y_val)):
    Y_val_label[i, Y_val[i]] = 1
    
#test
Y_test_label = np.zeros((len(Y_test), 256))
for i in range(len(Y_test)):
    Y_test_label[i, Y_test[i]] = 1

In [6]:
#Output dim for your dataset
output_dim = 256 #For Caltech256

#Data
coll = io.ImageCollection(X_train)
X_train_data = convert_image(coll)
# coll_val = io.ImageCollection(X_val)
# X_val_data = convert_image(coll_val)

In [14]:
#Model
tl_model = getModel( output_dim )
# rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # error
# sgd = keras.optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=True)
tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [15]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    samplewise_center=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_data)

In [17]:
hist = tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=20, 
             verbose=1, validation_split=0.5,
             shuffle=None, class_weight=None, sample_weight=None)

Train on 4096 samples, validate on 4096 samples
Epoch 1/20
4096/4096 [==============================] - 81s - loss: 3.8339 - acc: 0.2957 - val_loss: 2.2533 - val_acc: 0.4937
Epoch 2/20
4096/4096 [==============================] - 83s - loss: 0.8774 - acc: 0.7847 - val_loss: 1.9407 - val_acc: 0.5574
Epoch 3/20
4096/4096 [==============================] - 90s - loss: 0.4238 - acc: 0.8972 - val_loss: 1.8641 - val_acc: 0.5764
Epoch 4/20
4096/4096 [==============================] - 90s - loss: 0.2102 - acc: 0.9526 - val_loss: 1.8527 - val_acc: 0.5908
Epoch 5/20
4096/4096 [==============================] - 89s - loss: 0.1063 - acc: 0.9795 - val_loss: 1.8598 - val_acc: 0.5972
Epoch 6/20
4096/4096 [==============================] - 89s - loss: 0.0559 - acc: 0.9924 - val_loss: 1.8698 - val_acc: 0.6035
Epoch 7/20
4096/4096 [==============================] - 84s - loss: 0.0313 - acc: 0.9971 - val_loss: 1.8784 - val_acc: 0.6082
Epoch 8/20
4096/4096 [==============================] - 83s - loss: 0.

In [18]:
print(hist.history)

{'acc': [0.295654296875, 0.78466796875, 0.897216796875, 0.95263671875, 0.9794921875, 0.992431640625, 0.9970703125, 0.99853515625, 0.998779296875, 0.9990234375, 0.9990234375, 0.999267578125, 0.999267578125, 0.999267578125, 0.999267578125, 0.999267578125, 0.999267578125, 0.999267578125, 0.999267578125, 0.999267578125], 'loss': [3.8339065322652459, 0.87737164075952023, 0.42384960892377421, 0.21017872382071801, 0.10634218835912179, 0.055873461802548263, 0.031270003280951641, 0.019361631479114294, 0.013522462242690381, 0.010386978488895693, 0.0085830848147452343, 0.0074905256324200309, 0.0067942774112452753, 0.0063286272265941079, 0.0060036300865249359, 0.0057683979184730561, 0.0055927917755980161, 0.0054582444392963225, 0.0053528656587786827, 0.0052687619411244668], 'val_acc': [0.49365234375, 0.557373046875, 0.576416015625, 0.5908203125, 0.59716796875, 0.603515625, 0.608154296875, 0.61083984375, 0.615478515625, 0.616943359375, 0.619140625, 0.62060546875, 0.622314453125, 0.625, 0.6262207031